# Homework 1 - Slow general algorithms for sequence labeling

For homework 1, we will mainly be looking at a contrived task described below.  In this first homework, you will use slow but general algorithms.  They could be applied to *any* finite sequence problem, since they do not take advantage of any special problem structure.

The coding problems in this assignment are designed to be fairly small, with each taking around **5-20 lines of code**.  Throughout this assignment you will see parameters `xx`, `yy`, `oo`, `aa`, which correspond to  $\mathbf{x},
\mathbf{y}, \mathbf{o}, \mathbf{a}$ from [the formalisms document](https://seq2class.github.io/scribe-notes/formalisms.pdf): the doubled letter is meant to suggest that the variable represents a string.  So `xx` is the input string, `yy` ranges over possible output strings, `oo` is a (partial) observation of the output string, and `aa` ranges over over possible decisions (predictions or other plans) that our system can choose.

There's an awful lot of code here to read!  But this is getting you set for future assignments, where we will make gradual extensions to this codebase.  Understanding the class design will also reinforce your understanding of the concepts in the formalisms document.  Finally, you might learn something here about Python and object-oriented design.

## Our applied task

For this assignment we are predicting which vowels in a word are stressed.
In the (made-up) natural language used in our dataset, every vowel sound is 
represented by one of the letters `aeiou`, and we will use `AEIOU`
to indicate their stressed versions.

Following the notation defined so far, the input sequence $\mathbf{x}$ is our input
word (with no capitalization).  Each possible output $\mathbf{y}$ 
contains the same sequence of letters as $\mathbf{x}$, but with some of them 
`cApitalIzed` to indicate stress.  Therefore, if $\mathbf{x}$ contains $m$ vowels, 
then $|\mathcal{Y}_{\mathbf{x}}| = 2^m$.  

Sometimes this exponentially large search space gets reduced because for *some* vowels, 
we *observe* whether they are stressed or unstressed.  The observation is a string 
$\mathbf{o}$ that is a version of $\mathbf{y}$ with some of the vowels already given correctly 
as capital or lowercase, but unobserved vowels replaced with `?`.  So if 
$\mathbf{o}$ contains $n$ question marks (where $0 \leq n \leq m$), 
then $|\mathcal{Y}_{\mathbf{x,o}}| = 2^n$.

### Sample inputs and outputs
| $\mathbf{x}$ | $\mathbf{o}$ | $\mathbf{y} \in \mathcal{Y}_{\mathbf{x},\mathbf{o}}$ | $\mathbf{y} \notin \mathcal{Y}_{\mathbf{x},\mathbf{o}}$ (illegal)|
|:-------------:|:------------:|:---------------------:|:--------------------|
| `helilela` | `h?lil?l?` | `helilela` | `Helilela` |
| | | `helilelA` | `helIlela` |
| | | `hElilelA` | ... |
| | |    ...     |     |
| `idonotul` | `?don?t?l` | `IdonotUl` | `idOnotUl` |
| | | ... | ... |

## Assignment Instructions

Follow through the assignment in order.  Each text cell will provide details about the task and explain what is expected.

### Code

Inside of code blocks, you will see `### STUDENTS START` to indicate where you are expected to edit the cell and fill in some code (until `### STUDENTS END`).
Do NOT create new cells to insert code -- these will not be registered by the autograder!

### Writing

There are also a few short-answer questions throughout the assignment.
These are marked in red with <span style='color:red'>FILL IN</span>; again, edit the cell and only this cell to give your answer.

You can search for the phrases `STUDENTS START` and `FILL IN` to see if you have any unfinished work.  You will hand this notebook in via Gradescope -- more details on this will be posted on Piazza.

In [ ]:
import numpy as np
import scipy.optimize
from collections import namedtuple
import csv
import sys
import re

Data_type = namedtuple('Data', ['xx', 'oo', 'yy'])

def iterate_data(filename='train', *, max_examples=None):
    file = open(filename+'.tsv')
    for n, row in enumerate(csv.DictReader(file, delimiter='\t')):
        if max_examples is not None and n >= max_examples:
            break
        yield Data_type(
            xx=tuple(row['xx']),
            oo=tuple(row['oo']) if 'oo' in row else None,
            yy=tuple(row['yy']) if 'yy' in row else None,
        )

## Let's start by looking at the data ...

Our *strings* are sequences of symbols from our alphabet, represented as Python tuples.  Although our symbols in this problem are letters, they might be words in future problems, which is why we "convert" them into Python tuples of characters (python's duck typing makes it all still work out if you don't though (as you can still access the n-th character the same way) -- and it's sometimes less visual clutter -- so sometimes we omitted the `tuple()` call).

Each example consists of a triple `(xx,oo,yy)` where the observation string `oo` may include the special symbol `?`.
* A **training example** may specify either 
  * `oo=None` and `yy` is the fully observed output
  * `yy=None` and `oo` is a partial observation of the output (see formalisms.pdf, "Observations")

In [ ]:
itr = iterate_data()   # uses training set by default
print(next(itr))       # look at the first example in our training set
next(itr)              # look at the second example in our training set

* A **test example** consists of a triple `(xx,oo,yy)`, where `yy` is the correct answer.
  The prediction rule will be given `(xx,oo)` and asked to predict `yy`.
  * Usually `oo=None`, so the rule must simply predict `yy` from `xx`.  
  * However, if `oo` is specified, this informs the predictor that the true `yy` is compatible with `oo`.  This extra information should improve the prediction.
  <br><br>
* A **dev example** has the same format as a test example, since the development set is a test set that you use for practice.

In [ ]:
itr = iterate_data('dev')   # okay to peek at the development dataset (but not the real test data!)
print(next(itr))            # oo as partially observed output
next(itr)

## Task settings

Here is the base interface that we will be extending for the remainder of this homework.
All of our methods are defined with a `*` argument, like this:
```python
def mymethod(self, *, xx, oo=None, yy):
    pass   
```
The `,*,` prevents the parameters `xx, oo, yy` from being passed as positional arguments; instead they must be passed as keyword arguments.  This makes the code easier to read, and prevents errors due to accidentally specifying the arguments in the wrong order.


A `TaskSetting` object defines the setup for our decision or prediction task.  Its `iterate_yy` method defines the output spaces $\mathcal{Y}_{\mathbf{x}}$ and $\mathcal{Y}_{\mathbf{x},\mathbf{o}}$, while its `iterate_aa` method defines the action space $\mathcal{A}_\mathbf{x}$.  Its `reward` method defines the the environment's reward function $R(\mathbf{a} \mid \mathbf{x}, \mathbf{y})$ that returns how good our prediction $a$ was given the true answer is $y$.  For now, just read the code; we will start implementing the methods on this class in the [first problem](#Problem_stresstask).
<a id="TaskSetting"></a>

In [ ]:
class TaskSetting(object):
    """
    Base interface for specifying a task.
    Defines the output space, the action space, and the reward function.
    """
    
    def iterate_yy(self, *, xx, oo=None):
        """
        Returns an iterator over legal `yy` sequences (represented as tuples).
        If an observation `oo` is specified, restricts to `yy` sequences that 
        are consistent with `oo`.
        This method *defines* the space of output strings that we will consider 
        (although some of those could turn out to have probability 0).
        It also *defines* how `oo` is to be interpreted.
        The default implementation generates each `yy` one character at a time,
        by calling `iterate_y`.
        
        Caveat to users: Some implementations (for efficiency), instead of yielding
        a stream of immutable tuples, might keep yielding mutated versions of 
        the *same* list object.  Thus, you should print, analyze,
        or copy the `yy` values as you iterate through them.  Don't write `list(iterator)` 
        since that might give a list of many pointers to the *same* object; to save all
        values in a list, you should instead write `[tuple() for yy in iterator]`.
        """
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
        
    def iterate_y(self, *, xx, oo=None, yy_prefix):
        """
        Returns an iterator over legal next characters of `yy`.
        In other words, returns all characters `y` such that
        the concatenation `y_prefix + y` is a prefix of some
        output `yy` that is legal given input string `xx` 
        and observable `oo`.
        (This set of characters is discussed in formalisms.pdf, 
        "Restricting summations to the output space".)
        
        How do we know when we have reached the end of `yy`?  
        For this course, you can just assume `len(yy)=len(xx)`.  
        However, a more general design is to use `None` as an EOS
        symbol.  If you prefer that design, then `iterate_y`
        should also yield `None` if `y_prefix` can itself serve 
        as a legal output `yy`.
        """
        raise NotImplementedError()
        
    def iterate_aa(self, *, xx):
        """
        Returns an iterator over plans that are allowed for input `xx`.
        The default implementation just calls `iterate_yy(xx=xx)`, which is 
        appropriate for prediction tasks where the plans simply correspond 
        to predicting the different outputs.  This can be overridden for
        other kinds of decision tasks.
        (See formalisms.pdf, "Decision theory" and "More decision theory".)
        """
        yield from self.iterate_yy(xx=xx)
        
    def reward(self, *, aa, xx, yy):
        """
        Return the reward that plan `aa` will get on input `xx` if the true answer is `yy`.
        This method *defines* the reward function.
        """
        assert yy is not None    # should appear in subclass implementations too 
        raise NotImplementedError()
        
    def reward_threshold(self, *, xx):
        """
        Return a value `t` such that we consider plans with reward >= `t` to be "good"
        and plans with reward < `t` to be "errors".  This can be used for listing errors
        and serves as additional documentation of the reward function.  Note that the
        threshold may depend on `xx`.
        """
        raise NotImplementedError()

<a id="Problem_stresstask"></a>
## The vowel stress task
Now let's instantiate that [`TaskSetting`](#TaskSetting) interface for our task of stressing vowels:  
  * First fill in the function `StressTask.iterate_y` to iterate through the domain of $\mathbf{y}_t$ conditioned on the input $\mathbf{x}, \mathbf{o}$ and the prefix of the y string $\mathbf{y}_{0:t-1}$.  Test that it works.  
  * Then go back to the parent class above and fill in `TaskSetting.iterate_yy`, so that it uses `iterate_y` to iterate through the entire domain of $\mathcal{Y}_{\mathbf{x}, \mathbf{o}}$.  Sometimes the parameter `oo` will be `None` indicating that we instead want to iterate the domain of $\mathcal{Y}_\mathbf{x}$
  
*Python hints*: Some useful Python constructions are illustrated in the provided code below.  You can define an iterator in Python using `yield` as follows:
```python
def f():
    yield 'a'
    yield 'e'
```
Now you can write expressions like `list(f)` to build a list of the iterates, or write loops like this:
```python
for vowel in f():
    print(vowel)
```
You may also find the `yield from` operator to be useful (look it up online!).

In [ ]:
class StressTask(TaskSetting):
    """
    Class of models for the vowel stress problem, with
    a simple 0-1 reward function.
    """
    
    def reward(self, *, aa, xx, yy):
        assert yy is not None
        return 1 if yy == aa else 0    # was the answer exactly right?
    
    def reward_threshold(self, *, xx):
        return 1
        
    def iterate_y(self, *, xx, oo=None, yy_prefix):
        """
        Iterate through the domain of y_t given xx, oo, yy_{0:t-1}.
        """
        t = len(yy_prefix) 
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
   
    def iterate_yy(self, *, xx, oo=None):
        # Assert that the observable (if any) has the right format.
        # (This is more efficient than checking within `iterate_y`,
        # which would do the same checks repeatedly.)
        if oo is not None:
            assert len(oo) == len(xx)
            for t in range(len(oo)):
                assert oo[t] == xx[t] or (xx[t] in 'aeiou' 
                                          and (oo[t] == '?' or oo[t] == xx[t].upper()))
        # Now just invoke the parent class's default `iterate_yy`, 
        # which calls our specialized `iterate_y`.
        yield from super().iterate_yy(xx=xx, oo=oo)

task = StressTask()

Now, using that class, let's check that your `iterate_yy` correctly iterates over possible output strings given $\mathbf{x}$ and optionally $\mathbf{o}$.  We give you two test cases here to get started.  You should add your own test cases as well to this notebook cell.  

(*Note*: Python uses `t = tuple(s)` and `s = ''.join(t)` to convert between a string `s` and a tuple `t` of its characters.)

In [ ]:
assert set(task.iterate_yy(xx=tuple('test'))) == {tuple('test'), tuple('tEst')}
[''.join(yy) for yy in task.iterate_yy(xx=tuple('testphrase'), oo=tuple('t?stphrAs?'))]
### STUDENTS START
raise NotImplementedError()  # REPLACE ME
### STUDENTS END


In [ ]:
for xx, oo, yy in iterate_data('dev', max_examples=5):
    aa = task.iterate_aa(xx=xx)
    task.reward(aa=aa, xx=xx, yy=yy)
    print(''.join(list(aa)[3]))

## Probability models
To choose among the outputs `yy`, we'll want to make use of a probability model.

A `ProbabilityModel` object specifies a conditional probability distribution $p_{\boldsymbol{\theta}}(\mathbf{y} \mid \mathbf{x})$ with parameter vector $\boldsymbol{\theta}$.  The object has methods to evaluate probabilities, to sample from the distribution, and to train its parameters.  

In order to compute probabilities $p(\mathbf{y} \mid \mathbf{x})$ or to sample from the distribution, the object may need to be able to enumerate the domain $\mathcal{Y}_{\mathbf{x}}$.  That domain is specified by some instance of `TaskSetting`, which must be passed to the constructor and gets stored in `self.task`.

The subclass `BoltzmannModel` specializes `ProbabilityModel`.  It uses $\boldsymbol{\theta}$ to define a `score` function that corresponds to $G_\boldsymbol{\theta}(\mathbf{x}, \mathbf{y})$, and then takes $p_{\boldsymbol{\theta}}(\mathbf{y} \mid \mathbf{x}) \propto \exp G_\boldsymbol{\theta}(\mathbf{x}, \mathbf{y})$ — a Boltzmann distribution.

Below we give the general interfaces.  For now, just read the code.  You will soon come back and fill in the `BoltzmannModel.prob` method (as well as `BoltzmannModel.normalizer`, which it calls).  Don't worry about `logprob_gradient` or `sample` yet: you'll fill those in even later in the assignment.

The specific parameters and scoring method are not implemented in these generic classes.  We'll fill them in [below](#Problem_loglinstressmodel) in a subclass that defines a *particular* family of Boltzmann distributions — i.e., a particular probability model for our task.
<a id="ProbabilityModel"></a>

In [ ]:
class ProbabilityModel(object):   
    """
    Base class for conditional probability models P_theta(y | x).
    """
    
    def __init__(self, task):
        assert isinstance(task, TaskSetting)
        self.task = task
        self.initialize_params()
    
    def initialize_params(self):
        """
        Reset the model parameters to their start state.
        """
        raise NotImplementedError()
       
    @property
    def params(self):
        """
        Return a copy of the parameter vector for this model.
        """
        raise NotImplementedError()
    
    @params.setter
    def params(self, new_param_vec):
        """
        Update the parameters for the model to equal new_param_vec.
        """
        raise NotImplementedError()
    
    def prob(self, *, xx, oo=None, yy=None):
        """
        Return p(yy | xx) or p(oo | xx).  Only one of `yy` or `oo` should be specified.
        
        If `yy` is not a legal string in self.task's output space, or `oo` is not a
        legal observable, then we would ideally raise an error, but you are not 
        required to implement that.
        """
        assert (oo is None) != (yy is None)   # should appear in subclass implementations too
        raise NotImplementedError()
        
    def uprob(self, *, xx, oo=None, yy=None):
        """
        Just like `prob`, except that this version is free to return an 
        unnormalized probability when that is more efficient.
        The default implementation just calls `prob`.
        """
        return self.prob(xx=xx, oo=oo, yy=yy) 
        
    def logprob_gradient(self, *, xx, oo=None, yy=None):  
        """
        The gradient of log p(yy | xx) or log p(oo | xx) with respect to the 
        model parameters `params`, evaluated at the current model parameters.
        
        Either `yy` as a fully observed output or `oo` as a partial observation
        should be specified, but not both. (See formalisms.pdf, "Observations".)
        
        This is typically used to help estimate the parameters of the model.
        """
        assert (oo is None) != (yy is None)   # should appear in subclass implementations too
        raise NotImplementedError()
        
    def logprob_per_example(self, dataset):
        """
        Return the log of the conditional probability assigned
        by the model to an example, averaged over all examples
        in the given dataset.  This is useful to check the predictive 
        power of the model `p(yy | xx)`.  
        
        For each example, this method will sum `log p(yy | xx)` if 
        `yy` is defined, and otherwise `log p(oo | xx)`.  It never 
        conditions on `oo`, since the model is intended to capture
        `p(yy | xx)`.
        
        On a training dataset, this measures log-likelihood (how well
        the model fits the training examples).  On a dev or test dataset
        it measures held-out log-likelihood (how well the model predicts
        held-out examples).
        
        The default implementation calls `prob`.
        """
        total_logprob = 0
        count = 0
        for xx, oo, yy in dataset:
            total_logprob += np.log(self.prob(xx=xx, oo=oo) if yy is None 
                                    else self.prob(xx=xx, yy=yy))
            count += 1
            if count % 50 == 0: sys.stdout.write('\r\tevaluated log-probability on {} examples'.format(count))
        sys.stdout.write('\n')
        return total_logprob / count
    
    def sampler(self, *, xx, oo=None, temperature=1):
        """
        Generates an infinite stream of samples `yy` drawn exactly
        from p(`yy` | `xx`) or p(`yy` | `xx`, `oo`).
        The default method uses brute force via `self.task.iterate_yy`. 
        """
        # Note: We return a stream to avoid redoing work when we want *many* samples.
        # You should only have to compute the unnormalized probabilities and the 
        # normalizer once and then keep reusing them for the whole stream.
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
        
    def approx_sampler(self, *, xx, oo=None, temperature=1):
        """
        An approximate version of `sampler`.
        The default method uses Gibbs sampling, so successive samples in the
        stream will be correlated.
        
        Caveat to users: Same caveat as at TaskSampler.iterate_yy.
        """
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END


<a id="BoltzmannModel"></a>

In [ ]:
class BoltzmannModel(ProbabilityModel):
    """
    Base class for conditional probability models of the form
    P_theta(y | x) = (1/Z(x)) exp (G_theta(x,y) / T),
    that is, a Boltzmann distribution with temperature T.
    We refer to G_theta(x,y) as a "score".
    """
    
    def score(self, *, xx, yy):
        """
        Return the score G(`xx`, `yy`) as defined by the current params.
        By default, call `score_with_gradient` and only return the score.
        """
        score, gradient = self.score_with_gradient(xx=xx, yy=yy)
        return score
        
    def score_with_gradient(self, *, xx, yy):
        """
        Return two values: the score G(`xx`,`yy`) and its gradient with respect to the params.
        It's often convenient to compute the gradient along with the score, and we'll sometimes
        need the gradient.
        This method usually *defines* G.
        """
        raise NotImplementedError()
  
    def normalizer(self, *, xx, oo=None, temperature=1):
        """
        The normalizing function `Z(xx)` or `Z(xx,oo)`, 
        often called the "partition function", that is used to define
            p(yy | xx)     = \frac{1}{Z(xx)}    exp G(...)
            p(yy | xx, oo) = \frac{1}{Z(xx,oo)} exp G(...)
        (See formalisms.pdf, "Marginal and conditional probabilities".)
        
        The default implementation computes this by a brute-force sum with `iterate_yy`.
        However, that could be overridden by a more efficient method when available, 
        or by an approximation.
        """
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
        
    def uprob(self, *, xx, oo=None, yy=None, temperature=1):
        assert (oo is None) != (yy is None)
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
        
    def prob(self, *, xx, oo=None, yy=None, temperature=1):
        assert (oo is None) != (yy is None)
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
        
    def logprob_gradient(self, *, xx, oo=None, yy=None):
        assert (oo is None) != (yy is None)
        # Warning: Don't inadvertently recompute the same normalizer many times!  That's inefficient.
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END


<a id="Problem_loglinstressmodel"></a>
## Building a model for the stress task

We will design a simple linear scoring function $G_\boldsymbol{\theta}(\mathbf{x}, \mathbf{y})$.  This means that our probability model will be log-linear.

Our class `LoglinearStressModel` will define a simple linear scoring function $G_\boldsymbol{\theta}$, using hand-written features whose weights are specified by the parameter vector $\boldsymbol{\theta}$.
Its `score` method can return a score $G_\boldsymbol{\theta}(\mathbf{x},\mathbf{y}) \in \mathbb{R} \cup \{ -\infty \}$, so you could write code like this:
```python
from math import inf 
model = LoglinearStressModel(task)
assert model.score(xx=xx, yy=yy) == -inf
```

An instance of the class specifies a particular value for the parameters.  Its parameter vector $\boldsymbol{\theta}$ can be examined via the `params` property, and should be updated like this:
```python
update = np.zeros_like(model.params)  # a zero vector with the same dimensionality as params
update[7] = 123                       # put some nonzero elements into that vector
update[8] = 456
model.params += update                # add it to params (+= makes Python invoke set_params)
```
This may seem a bit roundabout: why not just modify `params` directly via statements like `model.params[7] += 123`?  The reason is that the object does not actually have a `params` attribute.  Its parameters are stored in one or more attributes with other names.  Calling `params` invokes a "getter" function (the `params` property) that constructs a new vector of all those parameters.  Modifying that new vector would not change the original parameters, whereas calling `params += ...` invokes a "setter" function that does so.

To make clear which parts of the definition are shared by all log-linear models, we start by defining a generic `LoglinearModel` class with a simple feature vector, which may be useful later.  Our `LoglinearStressModel` is a *particular* subclass — that is, a particular linear scoring model.  It could be further subclassed to give richer models with more features.  The  brute-force methods on this assignment make no assumption about what $G$ looks like, so you would be free to define $G$ using crazy features ("is the number of stressed vowels prime?" "what is the score of `yy` according to a certain neural network?").

In [ ]:
class LoglinearModel(BoltzmannModel):  
    """
    A base class for log-linear models: 
    These are just Boltzmann models with linear scoring functions,
    so we inherit from BoltzmannModel.
    We assume that the parameters can be stored in an attribute `_theta`.
    The training method is still not defined here.
    """

    def __init__(self, task):
        super().__init__(task)
        self.initialize_params()

    def initialize_params(self):
        raise NotImplementedError()
        
    @property
    def params(self):
        return np.array(self._theta)

    @params.setter
    def params(self, val):
        assert np.isfinite(val).all()
        self._theta[:] = val

    def features(self, *, xx, yy):
        """Extract a feature vector that measures various features of the pair (xx,yy)."""
        raise NotImplementedError()
        
    def score_with_gradient(self, *, xx, yy):
        """
        Return two values: the score G(`xx`,`yy`) and its gradient with respect to the params.
        It's convenient to compute the gradient along with the score, and we'll need it later.
        """
        # The score is the dot product of params theta with the feature vector,
        # which implies that its gradient is just the feature vector.
        f = self.features(xx=xx, yy=yy)
        score = self._theta.dot(f)    
        return float(score), f
    

In [ ]:
class LoglinearStressModel(LoglinearModel):
    
    """
    A specific log-linear model for StressTask.
    """   
    def __init__(self):
        super().__init__(StressTask())   # specify which task this model is for
    
    def initialize_params(self):
        np.random.seed(42)
        self._theta = np.array([
            .1,
            .05,
            0,
            1,
            0,
        ])

    def features(self, *, xx, yy):
        string = ''.join(yy)                          # concatenate symbols into an ordinary python string
        vowels = re.sub(r'[^aeiouAEIOU]','', string)  # extract just the vowels
        
        # All of our features are counts of structures in `(xx,yy)`.
        # For this problem, the features don't need to look at `xx`, but features for a POS tagger would do that.
        uppercase_vowels = len(re.findall(r'[AEIOU]', vowels))
        altcase_vowels = len(re.findall(r'(?=([aeiou][AEIOU])|([AEIOU][aeiou]))', vowels))
        enduppercase_vowels = len(re.findall(r'[AEIOU]$', vowels))
        uppercase_consonants = len(re.findall(r'[TNSRHDLC]', string))  
        length = len(string)

        # Assemble those counts into a feature vector.
        return np.array([uppercase_vowels,
                         altcase_vowels,
                         enduppercase_vowels,
                         uppercase_consonants,
                         length])


model = LoglinearStressModel()

Let's try the model out with its default initial parameter setting.  
What is each feature doing in the cell below?

**Answer:** <span style='color:red'>FILL IN</span>

Why don't the fourth and fifth features help to discriminate among the `yy` candidates?  Will that be true for every `xx`, or just for `xx=testphrase`?

**Answer:** <span style='color:red'>FILL IN</span>

To understand the features, it will probably help to experiment, so you may want to add your own examples to the cell below.

In [ ]:
testphrase = tuple('testphrase')
[( ''.join(yy),                                   # string yy
   model.features(xx=testphrase, yy=yy),   # yy's feature vector with this xx
   model.score(xx=testphrase, yy=yy)  )    # yy's score with this xx, using the current parameters 
 for yy in task.iterate_yy(xx=testphrase)]

Now go back to [BoltzmannModel](#BoltzmannModel) and fill in the `normalizer` and `prob` methods.

Let's check your implementation.  
Using the scores above, give simple numerical expressions (which may refer to $Z$) for:

$p(\mathbf{Y}=\texttt{testphrAsE} \mid \mathbf{X}=\texttt{testphrase}) =$ 

**Answer:** <span style='color:red'>FILL IN</span>

$p(\mathbf{Y} \in \texttt{testphrAs?} \mid \mathbf{X}=\texttt{testphrase}) =$

**Answer:** <span style='color:red'>FILL IN</span>

where $\texttt{testphrAs?}$ is being informally used to mean the *set* of output strings compatible with that partial observation.  
Those numerical expressions should match the answers computed below.

In [ ]:
Z = model.normalizer(xx=testphrase)
from math import isclose
assert isclose(Z, 9.812839692144227)  # checks that you got the right value

[Z, 
 model.prob(xx='testphrase',yy='testphrAsE'),
 model.prob(xx='testphrase',oo='testphrAsE'),  # should be the same as previous line
 model.prob(xx='testphrase',oo='testphrAs?')]  # should be bigger than previous line

Print out the whole distribution over `yy` for `xx='testphrase'` (using your iterator — this can be done in one line of Python).  
What happens if you raise or lower the temperature? Give some tests and discussion here:

In [ ]:
T = 1
### STUDENTS START
raise NotImplementedError()  # REPLACE ME
### STUDENTS END


**Discussion:** <span style='color:red'>FILL IN</span>

## Decision rules 
A `DecisionAgent` has a specific rule for making decisions in a given `TaskSetting` (namely `self.task`) by consulting a given `ProbabilityModel` (namely `self.model`).  As before, let's start with a general interface.
<a id="DecisionAgent"></a>

In [ ]:
class DecisionAgent(object):
    """
    Base class for the decision agents in this homework.
    
    The `decision` function in subclasses should implement some
    decision rule, which may refer to `self.task` (a `TaskSetting`)
    and `self.model` (a `ProbabilityModel`).
    """

    def __init__(self, task, model):
        """
        Arguments to the constructor are a TaskSetting 
        and a ProbabilityModel.
        """
        super().__init__()
        assert isinstance(task, TaskSetting)
        assert isinstance(model, ProbabilityModel)
        self.model = model
        self.task = task
    
    def decision(self, *, xx, oo=None):
        """
        Return some action `aa` that is appropriate to input `xx` and the partially
        observed output `oo` (if any).  
        
        This is the agent's decision rule.  It might make use of `model`, `reward`, 
        `iterate_aa`, and/or a random number generator.
        """
        raise NotImplementedError()
            
    def test(self, dataset):
        """
        Run the decision rule on all the examples in `dataset` 
        and return the average reward.
        """
        reward = 0
        count = 0
        for xx, oo, yy in dataset:
            aa = self.decision(xx=xx, oo=oo)
            reward += self.task.reward(aa=aa, xx=xx, yy=yy)
            count += 1
            if count % 50 == 0: sys.stdout.write('\r\tevaluated reward on {} examples'.format(count))
        sys.stdout.write('\n')
        return reward / count
    
    def show_errors(self, dataset, max_examples=20, reward_threshold=None):
        """
        Print (up to) `max_examples` examples in which the decision
        rule made a "bad" decision — one with reward < `reward_threshold`.
        `reward_threshold` may be a constant number or a function of the input `xx`,
        By default, it is the method task.reward_threshold.
        """
        # Modify reward_threshold if needed so that it's a function of `xx`
        if not callable(reward_threshold):
            if reward_threshold is None:
                reward_threshold = lambda xx: task.reward_threshold(xx=xx)
            else:  # it should be a constant number
                threshold = reward_threshold
                reward_threshold = lambda xx: threshold
                
        # Iterate over the data
        for xx, oo, yy in dataset:
            aa = self.decision(xx=xx, oo=oo)
            r = self.task.reward(aa=aa, xx=xx, yy=yy)
            if r < reward_threshold(xx):
                print(" r: {r}\n\tyy: {yy}\n\taa: {aa}\n\txx: {xx}\n\too: {oo}".format(
                     r=r,
                    xx=''.join(xx),
                    oo=''.join(oo),
                    yy=''.join(yy),
                    aa=''.join(aa),
                ))
                max_examples -= 1
                if max_examples == 0: break
                    
    ## The following methods are discussed later in the assignment.
    ## They ensure that decision agents are trainable.
                    
    def stochastic_gradient(self, **kwargs):
        """
        In general, a decision agent might have its own parameters, which
        it might train in such a way as to maximize reward.  (This is 
        particularly important in reinforcement learning.)
        
        By default, however, if the agent is asked to train on an example, it 
        will simply use the example to train the underlying probability model.
        """
        return self.model.stochastic_gradient(**kwargs)
        
    # By default, the params of the decision agent are the params of the underlying model.
    @property
    def params(self):
        return self.model.params

    @params.setter
    def params(self, val):
        self.model.params = val   

## The Viterbi decision rule
Our agent needs a decision rule.  Define a "Viterbi" decision rule that chooses the *most probable* `yy` given the input `xx`.  In the case of a Boltzmann model, that is also the `yy` that scores highest with `xx`:
$$\text{argmax}_{\mathbf{y} \in \boldsymbol{\mathcal{Y}}_{\mathbf{x}}} G_\boldsymbol{\theta}(x, y)$$

If `oo` is specified, the rule should choose the most probable `yy` that is *compatible* with `oo`:
$$\text{argmax}_{\mathbf{y} \in \boldsymbol{\mathcal{Y}}_{\mathbf{x},\mathbf{o}}} G_\boldsymbol{\theta}(x, y)$$

In [ ]:
class ViterbiAgent(DecisionAgent):

    def decision(self, *, xx, oo=None):
        """The Viterbi decision rule."""
        # Hint: Use self.task.iterate_yy and self.model.
        # Warning: Don't inadvertently recompute the same normalizer many times!  That's inefficient.
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END


Now let's make a Viterbi agent for our task, using our probability model.  Check that it is behaving as you expect, by trying out some more test cases in the following cell.

In [ ]:
viterbi_agent = ViterbiAgent(task, model)

viterbi_agent.decision(xx='testphrase')
### STUDENTS START
raise NotImplementedError()  # REPLACE ME
### STUDENTS END


Let's see what kind of predictions the agent makes on *development* data.  
(We haven't trained its parameters yet.)

In the code below, fill in assignments to `aa` (the prediction) and `reward` (the reward received as a result), so that the code prints out tuples $(R, \mathbf{x}, \mathbf{o}, \mathbf{y}, \mathbf{a})$.

In [ ]:
for xx, oo, yy in iterate_data('dev', max_examples=5):
    aa = reward = None  # replace with actual computation below
    ### STUDENTS START
    raise NotImplementedError()  # REPLACE ME
    ### STUDENTS END
    print((reward,
           ''.join(xx),
           ''.join(oo) if oo is not None else None,
           ''.join(yy) if yy is not None else None,
           ''.join(aa)))
    

Let's use the development dataset to check the overall quality of our predictions.
This returns the *average reward* over all development examples (so higher values are better).
Since we defined reward to be 1 if `aa==yy` and otherwise 0, our average reward is the *fraction* of examples where the agent predicted `yy` exactly.

In [ ]:
viterbi_agent.test(iterate_data('dev'))

Some of our "success" probably came from the fact that the dev data included partial observations $\mathbf{o}$ of the output $\mathbf{y}$, so the agent only had to pick the output from the set $\boldsymbol{\mathcal{Y}}_{\mathbf{x}, \mathbf{o}}$ of outputs that are compatible with $\mathbf{o}$.

How much worse would we have done without $\mathbf{o}$, so that we have to pick the right output from the larger  set $\boldsymbol{\mathcal{Y}}_{\mathbf{x}}$?

In [ ]:
# iterate over modified versions of the examples, in which oo is replaced by None
viterbi_agent.test([(xx,None,yy) for (xx,oo,yy) in iterate_data('dev')])

You may want to try out the `show_errors` method as well.

In [ ]:
viterbi_agent.show_errors(iterate_data('dev'), max_examples=5)

<a id="sup-training"></a>
## Gradients for supervised training

Our training objective below will include terms of the form $\log p_\boldsymbol{\theta}(\mathbf{y}_n \mid \mathbf{x}_n)$.  
Recall that in the case of a Boltzmann distribution (at temperature 1), we have
$$\begin{align*}
\log p_\boldsymbol{\theta}(\mathbf{y} \mid \mathbf{x}) 
&= \log \frac{\exp G_\boldsymbol{\theta}(\mathbf{x},\mathbf{y})}{Z(\mathbf{x})} \\
&= G_\boldsymbol{\theta}(\mathbf{x},\mathbf{y}) - \log Z(\mathbf{x})
\end{align*}$$
In general, the gradient of $\log Z$ will be the expected gradient of $G$.  Thus, the gradient of the above log-probability takes the classic form
$$\nabla_\boldsymbol{\theta} \log p_\boldsymbol{\theta}(\mathbf{y} \mid \mathbf{x}) 
= \nabla_\boldsymbol{\theta} G_\boldsymbol{\theta}(\mathbf{x},\mathbf{y}) 
   - \sum_{\mathbf{y'} \in \mathcal{Y}_{\mathbf{x}}}
      p_\boldsymbol{\theta}(\mathbf{y'} \mid \mathbf{x}) \nabla_\boldsymbol{\theta} G_\boldsymbol{\theta}(\mathbf{x},\mathbf{y'})$$
In the special case of a log-linear model, where $G_\boldsymbol{\theta}(\mathbf{x},\mathbf{y}) = \boldsymbol{\theta} \cdot \mathbf{f}(\mathbf{x},\mathbf{y})$, this reduces to the familiar expression *observed features minus expected features*,
$$\nabla_\boldsymbol{\theta} \log p_\boldsymbol{\theta}(\mathbf{y} \mid \mathbf{x}) 
= \boldsymbol{f}(\mathbf{x},\mathbf{y}) 
   - \sum_{\mathbf{y'} \in \mathcal{Y}_{\mathbf{x}}}
      p_\boldsymbol{\theta}(\mathbf{y'} \mid \mathbf{x}) \cdot \boldsymbol{f}(\mathbf{x},\mathbf{y'})$$
which you should recall from the [log-linear handout](https://www.cs.jhu.edu/~jason/tutorials/loglin/formulas.pdf), section 4.  We've merely generalized that derivation to non-linear $G$ functions by writing $\nabla G$ instead of $\boldsymbol{f}$.  It's really the same derivation.  

In the log-linear case, this log-probability is a concave function of $\boldsymbol{\theta}$; but with non-linear $G$ functions, it will generally suffer from local maxima.

*Exercise*: It was claimed above that for a Boltzmann distribution at temperature 1,
$$\nabla_\theta \log Z(x) 
   = \sum_{y' \in \mathcal{Y}_x} 
      p_\theta(\mathbf{y'} \mid \mathbf{x}) 
      \cdot \nabla_\theta G(x,y')$$
Prove it.

**Answer:** <span style='color:red'>FILL IN</span>

## Gradients for partially supervised training

More generally, if the $n$th training example is only partially observed, we replace $\mathbf{y}_n$ in the [supervised objective](#sup-training)
with the partial observation $\mathbf{o}_n$.  The modified term $p_\boldsymbol{\theta}(\mathbf{o}_n \mid \mathbf{x}_n)$ must sum over all possible values of $\mathbf{y}_n$, which is now a *latent variable*.  So the objective contains a log of a sum.  

We now have
$$\log p_\boldsymbol{\theta}(\mathbf{o} \mid \mathbf{x}) 
= \sum_{\mathbf{y} \in \mathcal{Y}_{\mathbf{x},\mathbf{o}}} p_\boldsymbol{\theta}(\mathbf{y} \mid \mathbf{x})
= \log \frac{Z(\mathbf{x},\mathbf{o})}{Z(\mathbf{x})} = \log Z(\mathbf{x},\mathbf{o}) - \log Z(\mathbf{x})$$

Partial observations (or equivalently latent variables) will generally introduce local maxima, even for a log-linear model.  Why?  We were lucky with the fully supervised version — a linear function minus a convex function is concave.  But the unsupervised log-linear objective is a convex function minus a convex function, which guarantees nothing.

Nonetheless, this generalization doesn't change our computations too much.
Remembering that the gradient of $\log Z$ is the expected gradient of $G$, we get
$$\nabla_\boldsymbol{\theta} \log p_\boldsymbol{\theta}(\mathbf{o} \mid \mathbf{x}) 
= \sum_{\mathbf{y} \in \mathcal{Y}_{\mathbf{x},\mathbf{o}}}
      p_\boldsymbol{\theta}(\mathbf{y} \mid \mathbf{x},\mathbf{o}) \cdot
        \nabla_\boldsymbol{\theta} G_\boldsymbol{\theta}(\mathbf{x},\mathbf{y}) 
   - \sum_{\mathbf{y'} \in \mathcal{Y}_{\mathbf{x}}}
      p_\boldsymbol{\theta}(\mathbf{y'} \mid \mathbf{x}) 
      \nabla_\boldsymbol{\theta} G_\boldsymbol{\theta}(\mathbf{x},\mathbf{y'})$$
The two terms in this expression arise from $Z(\mathbf{x},\mathbf{o})$ and $Z(\mathbf{x})$ respectively, so they are sometimes called the *numerator* and *denominator* terms.  
In the special case of a log-linear model, the expression is simply a difference of two expected feature vectors:
$$\nabla_\boldsymbol{\theta} \log p_\boldsymbol{\theta}(\mathbf{o} \mid \mathbf{x}) 
= \sum_{\mathbf{y} \in \mathcal{Y}_{\mathbf{x},\mathbf{o}}}
      p_\boldsymbol{\theta}(\mathbf{y} \mid \mathbf{x},\mathbf{o}) \cdot \boldsymbol{f}(\mathbf{x},\mathbf{y})     
   - \sum_{\mathbf{y'} \in \mathcal{Y}_{\mathbf{x}}}
      p_\boldsymbol{\theta}(\mathbf{y'} \mid \mathbf{x}) \cdot \boldsymbol{f}(\mathbf{x},\mathbf{y'})$$

Crucially, the two expectations are taken under different distributions: the "clamped" and "free" distributions.  
In the positive expectation, the random variable $\mathbf{Y}$ is (partially) "clamped" to match a complete observation $\mathbf{y}$ or a partial observation $\mathbf{o}$.  
In the negative expectation, the random variable $\mathbf{Y}$ is "free" to take on any value in $\mathcal{Y}_{\mathbf{x}}$.  
In both cases, the current model $p_\boldsymbol{\theta}$ is used to assign relative weights to the possible values of $\mathbf{Y}$.

At this point, you should go back to [BoltzmannModel](#BoltzmannModel) and fill in the `logprob_gradient` method, which can compute both fully and partially supervised gradients according to how it is called.  (If you prefer, you could implement only the fully supervised branch now, and fill in the other branch later.)  

Since the fourth and fifth features in `LoglinearStressModel` are not useful, the partial derivatives with respect to their weights should be about 0.  In general, the gradient should try to raise the weights of features whose values were observed to be larger in `yy`, or expected given `oo` to be larger in `yy`, than they are in the average candidate `yy`.

In [ ]:
[model.logprob_gradient(xx='testphrase',yy='testphrAsE'),
 model.logprob_gradient(xx='testphrase',oo='testphrAsE'),  # should be the same as previous line
 model.logprob_gradient(xx='testphrase',oo='testphrAs?')]  # should be different from previous line

## Stochastic gradient ascent

To train our probability model, we will tune $\boldsymbol{\theta}$ to maximize the *training objective* $$(\sum_{n=1}^N \log p_\boldsymbol{\theta}(\mathbf{y}_n \mid \mathbf{x}_n)) - c \cdot || \boldsymbol{\theta} ||^2$$
(or the more general version in which $\mathbf{o}_n$ replaces $\mathbf{y}_n$).
This objective is known as *L2-regularized conditional log-likelihood*.  The regularization coefficient $c > 0$ encourages keeping $\boldsymbol{\theta}$ close to $\mathbf{0}$.  

When using stochastic gradient ascent, it is useful to scale this down by a factor of $N$ (which doesn't change the optimal $\boldsymbol{\theta}$), so that our actual objective is
$$\begin{align*}
F(\boldsymbol{\theta}) 
&= (\frac{1}{N} \sum_{n=1}^N \log p_\boldsymbol{\theta}(\mathbf{y}_n \mid \mathbf{x}_n)) - \frac{c}{N} || \boldsymbol{\theta} ||^2 \\
&= \text{mean}_{n=1}^N F_n(\boldsymbol{\theta}) \\
\text{where }
F_n(\boldsymbol{\theta}) & = \log p_\boldsymbol{\theta}(\mathbf{y}_n \mid \mathbf{x}_n) - \frac{c}{N} || \boldsymbol{\theta} ||^2
\end{align*}$$

As a result, 
$\nabla_\boldsymbol{\theta} F(\boldsymbol{\theta}) 
= \text{mean}_{n=1}^N \nabla_\boldsymbol{\theta} F_n(\boldsymbol{\theta})$.  That is, the gradient of $F_n$ is, *on average*, equal to the gradient of $F$.  

Stochastic gradient ascent was taught in the reading handout for [NLP homework 3](https://www.cs.jhu.edu/~jason/465/hw-lm/hw-lm.pdf), section H.  You should review that handout; but the idea is that at each step, we'll choose a random example $n$ and nudge $\theta$ in the direction of its gradient $\nabla_\boldsymbol{\theta} F_n(\boldsymbol{\theta})$ — which is a good direction on average.  The forcefulness of this nudge (the "step size") decreases slowly over time.

The gradient on example $n$ is clearly
$$\nabla_\boldsymbol{\theta} F_n(\boldsymbol{\theta}) = \nabla_\boldsymbol{\theta} \log p_\boldsymbol{\theta}(\mathbf{y}_n \mid \mathbf{x}_n) - \frac{2c}{N} \boldsymbol{\theta}$$

### Implementing online training

Stochastic gradient ascent is an example of an "online" training algorithm — that is, it visits (randomly chosen) examples one at a time.  In our implementation, an object is "trainable" by an online gradient-based algorithm if it implements a `stochastic_gradient` method that can be given a single training example.  

So, let's equip our probability model with such a method.

(*Note:* Some other training algorithms are "batch" algorithms that consider the entire dataset at once.  We won't be using those, but here's how we could add them: an object would be trainable by a batch gradient-based algorithm if it implements a `batch_objective_with_gradient` method that can be given an entire dataset.)

It is traditional for optimization libraries to provide methods to *minimize* their functions, such as stochastic gradient *descent*.   To be compatible with such libraries, we will assume that a trainable object wants its objective function to be *minimized*.  So we will negate our actual objective function.

In [ ]:
class L2LogLikelihood:
    """
    This class can be mixed into a ProbabilityModel to 
    set a training objective of maximizing its L2-regularized
    log-likelihood, or equivalently, minimizing the negation
    of that.
    
    `regularization_coeff` is an attribute that can be modified
    directly and can also be specified by a keyword argument to
    the constructor.  The same is true for `num_examples`.
    """
    
    def __init__(self, *args, regularization_coeff=1, num_examples=None, **kwargs):
        assert regularization_coeff >= 0
        assert num_examples is None or num_examples >= 0
        self.regularization_coeff = regularization_coeff
        self.num_examples = num_examples
        super().__init__(*args, **kwargs)
    
    def batch_objective_with_gradient(self, dataset):
        raise NotImplementedError()     # not needed for this assignment, and probably not for this course

    def stochastic_gradient(self, *args, **kwargs):   
        """
        Note that num_examples must be specified for the stochastic 
        gradient case, so that the regularization term can be partitioned
        among all the training examples.
        """
        return -(self.logprob_gradient(*args, **kwargs) 
                 - (2 * self.regularization_coeff / self.num_examples) * self.params)

In [ ]:
# Multiple inheritance
class TrainableLoglinearStressModel(L2LogLikelihood, LoglinearStressModel):
    pass

# Redefine `model` to be a trainable version
model = TrainableLoglinearStressModel(regularization_coeff=15)

# Confirm that the model can compute the stochastic gradient of its training objective
model.num_examples = 1000    # must specify this in order to use stochastic_gradient method
[model.logprob_gradient(xx='testphrase',yy='testphrAsE'),
 model.stochastic_gradient(xx='testphrase',yy='testphrAsE')]   # will be different from previous line: 
                                                               # should be slightly modified by regularizer, 
                                                               # and also negated

Now we can define a general-purpose SGD training algorithm!

In [ ]:
from random import shuffle
class SGDTrainer(object):
    """
    Algorithm for training any object by stochastic gradient ascent,
    starting at its current parameters.  The object must implement a 
    `stochastic_gradient` method and have a `params` property.
    """
    def __init__(self, *, epochs=1, init_stepsize=0.05, decay=0):   # could add other convergence criteria
        self.epochs = epochs
        self.init_stepsize = init_stepsize
        self.decay = decay
    
    def train(self, trainable, dataset):
        iteration = 0              # count the number of updates so far
        dataset = list(dataset)    # make an internal copy
        print('\ttraining on dataset of {} examples'.format(len(dataset)))
        for _ in range(self.epochs):
            shuffle(dataset)       # permute in place so that the examples are visited in a random order
            for example in dataset:   
                example = dict(example._asdict())   # unpack named tuple into regular tuple
                stepsize = self.init_stepsize / (1 + self.init_stepsize * self.decay * iteration)   
                      # stepsize decreases slowly over time
                trainable.params -= stepsize * trainable.stochastic_gradient(**example)
                iteration = iteration+1
                if iteration % 50 == 0: 
                    sys.stdout.write('\r\ttrained for {} iterations'.format(iteration))  # print progress
        sys.stdout.write('\n')

Let's try out the whole thing, starting from scratch to see how the pieces are put together.  

Notice that we arranged for [DecisionAgent](#DecisionAgent) itself to be trainable — by default, it just passes the training examples along to train the probability model.

In [ ]:
# First, our task setting.
task = StressTask()

# Next, a training dataset for that task.  Let's store it in a list so we know how big it is.
trainset = list(iterate_data('train'))

# Now a conditional probability model with an L2-regularized 
# conditional log-likelihood objective that can be trained online.
class TrainableLoglinearStressModel(L2LogLikelihood, LoglinearStressModel):
    pass
c = 15
N = len(trainset)
model = TrainableLoglinearStressModel(regularization_coeff=c, num_examples=N)

# An agent for solving the task using the model.
viterbi_agent = ViterbiAgent(task, model)   # update agent to use the new version of the model
print("Average dev reward before training: {}".format(viterbi_agent.test(iterate_data('dev'))))

# A method for training the agent (which trains the underlying model).
trainer = SGDTrainer(epochs=2, decay=2*c/N)

# So let's train!
trainer.train(viterbi_agent, trainset)
print("Average dev reward after training: {}".format(viterbi_agent.test(iterate_data('dev'))))

You might sometimes want to train on a smaller dataset (check out the arguments to `iterate_data`).
It's interesting to see how performance changes as you reduce the amount of training data, and it's certainly faster while you get your code running. 

*Advanced note:* How did we choose the `decay` hyperparameter for SGD?  [Bottou (2012)](http://research.microsoft.com/pubs/192769/tricks-2012.pdf) says that the decay rate should be higher if the function is curvier.  We don't know the curvature of the objective function at the optimum, so I just used the curvature of the regularization term in the objective (where the curvature is the smallest eigenvalue of the Hessian at the optimum).

Check the parameters that the model learned.  Which ones appear to be most important for doing well on this task?

**Answer:** <span style='color:red'>FILL IN</span>

In [ ]:
viterbi_agent.model.params

Another question is how well we would have done in the semi-supervised training setting.  
`train-partial.tsv` provides a partially observed version of `train.tsv`, in which the `yy` column has been turned into an `oo` column.

This is the same setting for which the Expectation-Maximization algorithm is used.  Like EM, our training algorithm similarly tries to (locally) maximize the "incomplete-data log-likelihood" $\sum_{n=1}^N \log p_{\boldsymbol{\theta}}(\mathbf{o}_n \mid \mathbf{x}_n)$ (minus a regularizer).  But we are simply applying stochastic gradient directly to that objective. 

This is a popular alternative to EM.  In fact, it turns out that computing the stochastic gradient is essentially the same computation as the E step.  Both compute a vector of feature expectations under $p_\theta(\mathbf{y} \mid \mathbf{x},\mathbf{o})$.  They just use this vector a little differently within their overall maximization algorithms. 

In [ ]:
itr = iterate_data('train-partial') 
print(next(itr))            
next(itr)

In [ ]:
# Make a new model and train it on the partial version of the dataset
model2 = TrainableLoglinearStressModel(regularization_coeff=c, num_examples=N)  # new copy
viterbi_agent2 = ViterbiAgent(task, model2) 
print('Initial params: {}'.format(viterbi_agent2.params))

trainset2 = list(iterate_data('train-partial'))
# assert N == len(trainset2)

trainer.train(model2, trainset2)  # use the same trainer (hyperparameters, etc.)
print('Average dev reward after training: {}'.format(viterbi_agent2.test(iterate_data('dev'))))
print('Params after semi-supervised training: {}'.format(viterbi_agent2.params))

Let's look more systematically at what happens to the quality of the *model* (average log-probability of the truth) and the quality of the *agent* (average reward) when we train.  The systematic experiment below is run on a small amount of data, but you could play around and change that.

* Different lines below show what happens when we evaluate on examples from different datasets: `train`, `train-partial`, and `dev`. 
What do you learn by comparing these numbers?

**Answer:** <span style='color:red'>FILL IN</span>

* Does the trained probability model the training data well? (If not, we are "underfitting" the training data. We need a better model or less regularization.)

**Answer:** <span style='color:red'>FILL IN</span>

* Does training allow the model to generalize to held-out dev data well? (If we perform worse on dev data than on training data, then we are "overfitting" the training data. We need more regularization.)

**Answer:** <span style='color:red'>FILL IN</span>

* Did semi-supervised training (last section of the output) work as well as supervised training?

**Answer:** <span style='color:red'>FILL IN</span>

If you're curious, feel free to extend the experiment to look how performance is affected by other decisions: the regularization coefficient, the structure of your model, the amount of training data, the hyperparameters for the SGD method, random reruns of SGD, etc. If you choose to look into these questions, you can discuss here.

In [ ]:
def experiment(task, agent_classes=[], max_test_examples=None, max_train_examples=None):
    for i, train in enumerate(['train','train-partial']):
        
        trainset = list(iterate_data(train,max_examples=max_train_examples))
        model = TrainableLoglinearStressModel(regularization_coeff=c, num_examples=len(trainset))
        agents = [ constructor(task, model) for constructor in agent_classes ]
        
        def print_results():
            for test in ['train','train-partial','dev']:
                testset = list(iterate_data(test, max_examples=max_test_examples))
                results = (model.logprob_per_example(testset),
                           None if test=='train-partial'   # no answers there to score
                           else [agent.test(testset) for agent in agents])
                print('(logprob, rewards) per {} example = {}'.format(test, results))
                
        if i==0:  # for first version of train only, to avoid redundancy
            print('Model quality before training')
            print_results()
        print('\nTraining set: {}'.format(train))
        trainer.train(model, trainset)
        print('Model quality after training')
        print_results()

experiment(task, [ViterbiAgent], max_test_examples=300, max_train_examples=1000)

## A new reward function

So far we have been using a reward function that only rewards us when we are *exactly* right.  However, some task settings might offer "partial credit."

The *Hamming distance* between two strings of the same length is the number of positions in which they differ.  Our new reward function will be loosely based on Hamming distance between our prediction `aa` and the true string `yy`, with 3 changes:
* Hamming distance is really a *loss*, since larger distance is bad.  So we will negate it to get a reward.  
  Now the best possible reward is 0, and discrepancies between `aa` and `yy` are punished with *negative* reward.
* We make the function asymmetric.  An incorrectly stressed vowel (uppercase letter in `aa`) is punished with -1, 
  but an incorrectly unstressed vowel (lowercase letter in `aa`) is punished with -0.3.  Thus, it would be better 
  to err on the side of lowercase.
* We normalize by the length of `yy`.  Thus, no matter how long an example string is, its reward is 
  bounded between -1 and 0, just as it was formerly bounded between 0 and 1.

In [ ]:
a = LoglinearStressModel()
b = LoglinearStressModel()
a.params += 1
b.params += 2
print(a.params)
print(b.params)

In [ ]:
class HammingTask(StressTask):
    
    def reward(self, *, aa, xx, yy):
        assert yy is not None
        return sum(0 if aa[t] == yy[t] else (-.3 if aa[t] in 'aeiou' else -1.0) for t in range(len(aa))) / len(yy)
    
    def reward_threshold(self, *, xx):
        return 0    # best possible reward

hamming_task = HammingTask()

In [ ]:
hamming_task.reward(aa='tEst', xx='test', yy='test')

In [ ]:
hamming_task.reward(aa='test', xx='test', yy='tEst')

## Reward-infused decoding

Our old decision rule — the Viterbi decoder — tries to maximize the old reward (0 or 1).
Let's change the decision rule to try to optimize its prediction for the new (arbitrary) reward function instead.
This would be a "Bayes rule" if we knew the true probability distribution $p^*$:
 $$ \mathrm{argmax}_{\mathbf{a} \in \boldsymbol{\mathcal{A}}_{\mathbf{x}}} \sum_{\mathbf{y} \in \boldsymbol{\mathcal{Y}}_{\mathbf{x},\mathbf{o}}} p^*(\mathbf{y} \mid \mathbf{x}, \mathbf{o}) \cdot R(\mathbf{a} \mid \mathbf{x},\mathbf{y}) $$
 
Of course, we will have to use our trained probability model $p_\boldsymbol{\theta}$ instead of $p^*$.  
Go ahead and implement a brute-force version:

In [ ]:
class BayesAgent(DecisionAgent):
    """
    Try to make the decision that minimizes the Bayes risk 
    (or in positive terms, maximizes the Bayes value).
    """
    
    def decision(self, *, xx, oo=None):
        # Warning: Don't inadvertently recompute the same normalizer many times!  That's inefficient.
        ### STUDENTS START
        ### Bayes min risk decoding
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END


Test how well the Bayes agent performs with our Hamming distance reward.  Notice that the agent is simply consulting the `model` we've already trained and making predictions in a different way.  We don't have to retrain it.

In [ ]:
hamming_agent = BayesAgent(hamming_task, model)
hamming_agent.test(iterate_data('dev'))

How well would our Viterbi agent do on the same task, i.e., when evaluated on the new reward function?  Again, the model doesn't change, and the Viterbi rule makes the same decisions as before; we're just rewarding those decisions with a different formula.

In [ ]:
ViterbiAgent(hamming_task, model).test(iterate_data('dev'))

Notice that the Bayes agent is a lot slower.  That's because it has to maximize over all possible actions $\mathbf{a}$ in the outer loop, where each is evaluated by an expectation over all possible gold answers $\mathbf{y}$ in the inner loop.  As we'll see in the next assignment, for some reward functions the outer loop can be sped up by dynamic programming, and for some probability models the inner loop can be sped up by dynamic programming.

Feel free to look at the examples where the predictions are different (it's only a couple of lines of Python, right?).  We might expect that the Bayes agent is more conservative about adding stress, due to the asymmetric reward.  Or it might be that the Viterbi string ends with a capital letter, but most of the other strings end with a lowercase letter, so the Bayes agent thinks it's safer to pick lowercase.  

By the way, these new rewards look pretty good.  But maybe that's because they now include partial credit.  In fact, there's a large fraction of letters that we're never penalized on — we always get the consonants right, as well as the vowels that are observed in `dev.tsv`.  To know whether the results are actually impressive, we should compare with a simple baseline system.  For example, how much reward would the Viterbi or Bayes rule obtain with the default *initial* parameters of `LogLinearStressModel`?  Or with a uniform distribution?

In [ ]:
# experiment here with baselines!
### STUDENTS START
raise NotImplementedError()  # REPLACE ME
### STUDENTS END


## *"Crazy"* reward function
  
A task setting is free to define any reward function that compares the prediction $\mathbf{a}$ to the "gold" answer $\mathbf{y}$. One of the most common reward functions in natural language processing is BLEU.  BLEU is primarily used in machine translation. It checks whether the n-grams in $\mathbf{a}$ mostly appear in $\mathbf{y}$.  It can work when $\mathbf{a}$ and $\mathbf{y}$ are different lengths, so it includes a "brevity penalty" to prevent $\mathbf{a}$ from being very short and only including a few confident n-grams.  This reward function has become popular for scoring machine translation systems, as it has been shown to correlate with human preferences for translations, can be easily computed, allows "partial credit" when $\mathbf{a}$ is mostly good output, and allows $\mathbf{y}$ to be a set of good translations (written by different humans) instead of just one.

In this homework, we will be implementing a slightly simplified version of BLEU (the smoothing method is simpler, as shown at $\epsilon$ below).  BLEU computes the harmonic mean over $1 \leq n \leq 4$ of the precision of $\mathbf{y}$'s n-grams.  It's still rewarding to match the shorter n-grams even when we fail to match any of the longer n-grams.

$$
\begin{align*}
  \#\mathbf{y}_{\mathbf{c}} &:= \text{Number of instances of ngram $\mathbf{c}$ inside string $\mathbf{y}$} \\
  \text{ngram}_n &:= \text{set of ngrams $n$ tokens long} \\
  \epsilon &:= .01 \\
  p_n &= \frac{\max\{\sum_{\mathbf{c} \in \text{ngram}_n} \min\{\#\mathbf{a}_{\mathbf{c}}, \#\mathbf{y}_{\mathbf{c}}\}, \epsilon \} }{\max\left\{\sum_{\mathbf{c} \in \text{ngram}_n} \#\mathbf{y}_\mathbf{\mathbf{c}}, 1\right\}} & \textit{Precision with ``smoothing''} \\
  \textit{bp}(a, y) &= \left\{ \begin{array}{cc} 1 & \text{if }a \ge y \\ e^{1 - a/y} & \text{otherwise} \end{array} \right. & \textit{brevity penalty} \\
  R(\mathbf{a} \mid \mathbf{y}) &:= \text{exp}\left( \frac{1}{4} \sum_{n=1}^4 \log(p_n) \right) \cdot \textit{bp}(\text{len}(\mathbf{a}), \text{len}(\mathbf{y}))
  & \textit{BLEU score (reward function)}
\end{align*}
$$

In [ ]:
### Helper functions for bleu
from collections import Counter
def ngrams(n, string):
    grams = Counter()
    for i in range(0, len(string) - n + 1):
        grams[string[i:i+n]] += 1
    return grams

### Define more helper functions here ...
### STUDENTS START
raise NotImplementedError()  # REPLACE ME
### STUDENTS END

def bleu(*, aa, yy):
    ### STUDENTS START
    raise NotImplementedError()  # REPLACE ME
    ### STUDENTS END
    

class BleuStressTask(StressTask):
    
    def reward(self, *, aa, xx, yy):
        assert yy is not None
        return bleu(aa=aa, yy=yy)
    
    def reward_threshold(self, *, xx):
        return 1   # maximum possible reward

Sanity-check your BLEU implementation:

In [ ]:
bleu_task = BleuStressTask()
assert bleu(aa='test', yy='test') == 1
assert np.isclose(bleu(aa='TEST', yy='test'), .0045180100180492264)

### STUDENTS START
### Write additional tests for your bleu function
raise NotImplementedError()  # REPLACE ME
### STUDENTS END


How well does the agent do at BLEU?

In [ ]:
bleu_agent = BayesAgent(bleu_task, model)
bleu_agent.test(iterate_data('dev', max_examples=20))

## Speeding up the BLEU Implementation

It now appears that our decision rule has become somewhat slow after making it compute the BLEU function on all possible predictions `aa`.  When building more complicated models, we will often have to put our engineering hats on and figure out how to make our programs run faster.

First lets get a baseline for how fast our BLEU decoder is:

In [ ]:
%time bleu_agent.test(iterate_data('dev', max_examples=100))

Let's try and identify the slow functions using [`%prun`](http://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-prun).  This will generate a profile of all the functions that are called when running and how long each function takes to run.

In [ ]:
%prun bleu_agent.test(iterate_data('dev', max_examples=100))

Now that you have identified which functions are *slow* we are going to use [Cython](https://cython.readthedocs.io/en/latest/) to make our program faster by compiling the slow functions into C.

Cython uses Python-like syntax, but allows a few additional annotations, which allows Cython to compile a faster version.
     
#### Short Cython tutorial / hints
 1. In our jupyter notebook, we can load Cython using `%load_ext cython`.  We can then use Cython inside of any jupyter cell by putting `%%cython` at the top of the cell.  
 2. The parameter `%%cython -a` will turn on verbose mode which is helpful when trying to debug why our function is slow.  This will show the resulting C program that is generated from our Cython code.  Lines that are highlighted in <span style='background-color:yellow'>yellow</span> indicate that they are invoking a lot of python internals, and thus will be *generally slower*.
 3. Adding types to parameters can help reduce the overhead of calling a function.  Try both of these expressions in a `%%cython -a` block
 ```cython
%cython -a
 def multiply_by_2(a):
      return a * 2
 cdef float multiply_by_2(float a):
      return a * 2
 ```
 4. Defining types on local variables can also help.  Some types you could use include `dict`, `int`, `float`.  Defining a range parameter as `int` will help Cython generate a C-style `for` loop.  Using `dict` lets Cython omit some extra checks when accessing the elements inside of a dictionary type.
 ```cython
%cython -a
 cdef foo():
      cdef int i
      cdef dict d = {}
      for i  in range(10):
           d[i] = i
 ```
 5. We can replace `np.exp` with `libc.math.exp` which will use the `exp` function defined in C's `math.h` instead of making a slower call to numpy.
 

There is no *correct* answer to how fast your program should run.  You need to decide when your program is fast enough such that you can actually study the problems that you are interested in.  For this problem set, you should aim to get at least 2-3x faster that your baseline BLEU implementation.

In [ ]:
%load_ext cython

In [ ]:
%%cython -a

from libc.math cimport exp, log
import numpy as np
cimport numpy as np

### STUDENTS START
### Helper functions for bleu
raise NotImplementedError()  # REPLACE ME
### STUDENTS END

def faster_bleu(tuple yy, tuple aa):
    ### STUDENTS START
    raise NotImplementedError()  # REPLACE ME
    ### STUDENTS END
    

Check that our new BLEU function is predicting the same as our old function.  Also define a few additional checks of your own.

In [ ]:
assert bleu(yy=tuple('tEst'), aa=tuple('test')) == faster_bleu(yy=tuple('tEst'), aa=tuple('test'))

### STUDENTS START
raise NotImplementedError()  # REPLACE ME
### STUDENTS END


Check how much faster the fast bleu function is compared to the baseline bleu function

1. Baseline BLEU function runtime (seconds):

**Answer:** <span style='color:red'>FILL IN</span>

2. Faster BLEU function runtime (seconds):

**Answer:** <span style='color:red'>FILL IN</span>

3. Performance improvement (faster/baseline, %):

**Answer:** <span style='color:red'>FILL IN</span>

In [ ]:
class FasterBleuStressTask(BleuStressTask):
    
    def reward(self, *, aa, xx, yy):
        assert yy is not None
        return faster_bleu(aa=aa, yy=yy)
    
faster_bleu_task = FasterBleuStressTask()

faster_bleu_agent = BayesAgent(faster_bleu_task, model)
%time faster_bleu_agent.test(iterate_data('dev', max_examples=100))

Now that that our BLEU function is much faster, we can evaluate it over the entire dev set.  (Although really, 100 examples is often plenty for evaluation — just not for training.)

In [ ]:
faster_bleu_agent.test(iterate_data('dev'))

If you like, you could try including rerunning `experiment` from before on the BLEU task, and include the performance of the Viterbi, Hamming, and BLEU agents when measured by the BLEU reward.

## Expectations under $p_\boldsymbol{\theta}(\mathbf{y} \mid \mathbf{x})$

You may notice that we have used expected values in two ways:
* A Bayes rule evaluates a plan $\mathbf{a}$ for input $\mathbf{x}$
  according to its *expected reward*, 
  $$\mathbb{E}_{\mathbf{y}\mid\mathbf{x}}[R(\mathbf{a} \mid \mathbf{x},\mathbf{y})]
  = \sum_{\mathbf{y} \in \boldsymbol{\mathcal{Y}}_{\mathbf{x}}} p_{\boldsymbol{\theta}}(\mathbf{y} \mid \mathbf{x}) \cdot R(\mathbf{a} \mid \mathbf{x},\mathbf{y})$$
* The stochastic gradient of conditional log-likelihood from an example $(\mathbf{x},\mathbf{y})$ involves
  the term 
  $$\nabla_{\boldsymbol{\theta}} \log Z(\mathbf{x}) = \mathbb{E}_{\mathbf{y}\mid\mathbf{x}}[\nabla_{\boldsymbol{\theta}} G_{\boldsymbol{\theta}}(\mathbf{x},\mathbf{y})] = \sum_{\mathbf{y} \in \boldsymbol{\mathcal{Y}}_{\mathbf{x}}} p_{\boldsymbol{\theta}}(\mathbf{y} \mid \mathbf{x}) \cdot  \nabla_{\boldsymbol{\theta}} G_{\boldsymbol{\theta}}(\mathbf{x},\mathbf{y})$$

These are the main two uses of (conditional) expectations.  Most other uses are just variants on the two above — *prediction* from the model and *training* the model's parameters.  

(However, an additional use is to measure the model's posterior uncertainty about a quantity $f(\mathbf{y})$, given $\mathbf{x}$.  Common uncertainty measurements such as variance and entropy are actually expectations.)

### Estimating expectations by sampling from $p_\boldsymbol{\theta}(\mathbf{y} \mid \mathbf{x})$

So far, we have been computing expectations by actually summing over all $\mathbf{y} \in \mathcal{Y}_{\mathbf{x}}$.  But this is often intractable.  An alternative is to approximate an expectation by sampling. That is, $\mathbb{E}_{\mathbf{y}\mid\mathbf{x}}[f(\mathbf{y}]$ can be approximated as $\text{mean}_i f(\mathbf{y}_i)$ where the $\mathbf{y}_i$ are drawn (at least approximately) from $p_\theta(\mathbf{y} \mid \mathbf{x})$.  Many expectations can be well-estimated by averaging over a rather small sample. 

Later we will study various methods for sampling exactly or approximately.  

### Exact sampling

Sampling exactly means that if $p_\theta(\mathbf{y} \mid \mathbf{x}) = \frac{1}{4}$, then $\mathbf{y}$ really does have a $\frac{1}{4}$ probability of being drawn, so copies of $\mathbf{y}$ will make up $\frac{1}{4}$ of the sample on average.  Of course this doesn't mean that the sampling-based estimate of $\mathbb{E}_{\mathbf{y}\mid\mathbf{x}}[f(\mathbf{y})]$ will be correct — it will vary depending on your random number generator.  But as the number of samples $\rightarrow \infty$, the average error of the estimate provably decreases $\rightarrow 0$ (at least if $f(\mathbf{Y})$ has finite variance).

If exact sampling from a distribution can be done efficiently, it is usually just as efficient to compute the expectation exactly.  (For example, sampling and averaging may involve similar dynamic programming algorithms.)  Nonetheless, an exact sampler for an "easy" distribution might still be a useful ingredient in constructing an approximate sampler for a "harder" distribution.

As a reference implementation, go back and implement [ProbabilityModel](#ProbabilityModel).`sampler`.  as a brute-force exact sampler.  That is, you'll explicitly compute $p_{\boldsymbol{\theta}}(\mathbf{y} \mid \mathbf{x})$ for every $\mathbf{y} \in \mathcal{Y}_{\mathbf{x}}$ and then `yield` a stream of samples from that distribution.  You might find [np.random.choice](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.choice.html) useful.  Your implementation should also handle $p_{\boldsymbol{\theta}}(\mathbf{y} \mid \mathbf{x},\mathbf{o})$ if `oo` is specified.

In [ ]:
# Make sure you also reevaluate the notebook cells for `BoltzmannModel`, `LoglinearModel`, and `LoglinearStressModel`, 
# now that you've updated their ancestor `ProbabilityModel`.

model = LoglinearStressModel()             # an instance of the updated class
xx, oo, yy = next(iterate_data('train'))   # get the first training example

from itertools import islice     # lets us take the first N elements (or every Jth element) of an infinite stream
for yy in islice(model.sampler(xx=xx, oo=oo), 10): 
    print(yy)

So what is the expected number of stressed vowels in this example's output `yy`?

In [ ]:
def count_stresses(yy):
    "Count the number of capitalized characters (stressed vowels) in string `yy`."
    return len([y for y in yy if y.isupper()])

for _ in range(8):  # 8 different estimates, each based on 30 *exact* samples
   print(sum([count_stresses(yy) for yy in islice(model.sampler(xx=xx, oo=oo), 20)]) / 20)

### Approximate sampling

Enumerating the entire exponentially large domain $\mathcal{Y}_{\mathbf{x}}$ can be slow, so now let's implement an alternative: Gibbs sampling.  This could be done for any `ProbabilityModel`, so go back to [ProbabilityModel](#ProbabilityModel) and fill the `approx_sampler` method with a Gibbs sampler.  The Gibbs sampler is a pretty good default choice for subclasses to inherit.

The pseudo code for such a Gibbs sampler looks like this:
  * $\mathbf{y}$ = some initial legal guess of the output string (given $\mathbf{x}$)
  * Repeat forever:
    * Randomly choose a position $j$ in the output string
      * For each legal new value $y$ for $y_j$ 
        * temporarily set $y_j = y$
        * let $\tilde{q}(y) = \tilde{p}_\theta(\mathbf{y} \mid \mathbf{x})$
      * Draw $y$ from a normalized version of $\tilde{q}$ and set $y_j = y$
      * `yield` $\mathbf{y}$

with the obvious modifications to condition throughout on $\mathbf{o}$ if `oo` is specified.  

Note that the length of $\mathbf{y}$ never changes, which is okay since our setup assumes that all legal output strings have the same length.  In general, a Gibbs sampler only works on a fixed-length vector of random variables: $\mathbf{Y} = (Y_1,\ldots,Y_J)$.  (Going beyond this requires devising a fancier MCMC method for your specific problem.)  A common simplification is to repeatedly loop through $j = 1,\ldots,J$ rather than selecting $j$ randomly at each iteration.  

In any case, the algorithm above provides an iterator over an infinite stream of (non-IID) samples $\mathbf{y}_n: n=1,2,\ldots$.  As $n \rightarrow \infty$, the sampling distribution of $\mathbf{y}_n$ gets closer and closer to $p_\boldsymbol{\theta}(\mathbf{y} \mid \mathbf{x})$.

You can estimate $\mathbb{E}_{\mathbf{y}\mid\mathbf{x}}[f(\mathbf{y})]$ by averaging over
some of these samples — for example, all of the first $N$ samples (see [One Long Run](http://users.stat.umn.edu/~geyer/mcmc/one.html)), or perhaps only every $J$th or every $(5\cdot J)$th sample simply to reduce the
number of calls to $f$ (since nearby samples in the stream are similar to one another anyway).
Ideally, `ProbabilityModel` would provide an `expectation_approx` method that computes an 
approximate expectation in this way.  (As well as an exact `expectation` method that 
uses an exhaustive iterator over the exponentially large space $\mathcal{Y}_{\mathbf{x}}$.)

*Hints:* Use `iterate_yy` (or possibly `iterate_y`) to make your initial guess, and use `iterate_y` to propose modifications.  In the case of `StressTask`, the Gibbs sampler could be made more efficient, so feel free to override your implementation in `LoglinearStressModel`.  (In `StressTask`, $q$ is a distribution over at most two possibilities — capital and lowercase versions of $x_j$.  And for many positions $j$, there is just *one* possibility — if $x_j$ is a consonant or $o_j$ specifies the value of $y_j$ — so you don't have to visit that position $j$ at all.)

Check the behavior of `approx_sampler`.  Note how strongly correlated successive samples are, since at most one character is different.  This reduces the effective sample size — we're getting pretty much the same `yy` over and over again.

In [ ]:
for yy in islice(model.approx_sampler(xx=xx, oo=oo), 10): 
    print(yy)

To reduce the correlation between successive samples, let's try printing only every $J$th sample, where $J=|\mathbf{x}|$, so we are printing a new sample only after every "sweep" through all positions $j$.  This should look somewhat more like the exact sampler.

In [ ]:
J = len(xx)
for yy in islice(model.approx_sampler(xx=xx, oo=oo), J, 10*J, J): 
    print(yy)

In [ ]:
for _ in range(8):  # 8 different estimates, each based on 20 *approximate* samples (spaced J apart but still correlated)
   print(sum([count_stresses(yy) for yy in islice(model.approx_sampler(xx=xx, oo=oo), J, 20*J, J)]) / 20)

In [ ]:
for _ in range(8):  # 8 different estimates, each based on 200 *correlated* *approximate* samples
   print(sum([count_stresses(yy) for yy in islice(model.approx_sampler(xx=xx, oo=oo), 200)]) / 200)

### Comparing the samplers

In principle, you could use the approximate sampler to speed up the expectations used during training and when computing the decision rule.  

But to check that the exact and approximate samplers are doing roughly the same thing, let's estimate the Kullback-Leibler divergence between the distributions that they're sampling from.  The true distribution is $p_\boldsymbol{\theta}(\mathbf{y} \mid \mathbf{x})$.  The approximate sampler is implicitly drawing from some distribution $q(\mathbf{y} \mid \mathbf{x})$ that is harder to describe.  For a fixed $\mathbf{x}$, we can abbreviate these as $p_\boldsymbol{\theta}(\mathbf{y})$ and $q(\mathbf{y})$, and the KL divergence between these is
$$\text{KL}(p_\boldsymbol{\theta} \mid\mid q) = \mathbb{E}_{\mathbf{y} \sim p_\theta}[\log_2 \frac{p_\boldsymbol{\theta}(\mathbf{y})}{q(\mathbf{y})}]$$
This is clearly 0 if the two distributions are equal.

You can use brute force to compute the above expectation under $p_\boldsymbol{\theta}$.
To get the term $q(\mathbf{y})$, you should take a large sample from $q$ and count the fraction of times that $\mathbf{y}$ occurred.  Unfortunately, if your sample is too small, you might divide by 0.  Python's `Counter` class is useful for collecting these counts.

If your Gibbs sampler $q$ is working correctly, its KL divergence from $p_\boldsymbol{\theta}$ on your trained `LoglinearStressModel` should be $< .1$ bit on average (when conditioning on some `xx` or `(xx,oo)` from our dataset).  If you modify your code to replace the approximate sampler $q$ with the (possibly slower) exact sampler, then the (estimated) KL divergence should be $\approx 0$, which is a good test.

In [ ]:
def kl(model, *, xx, oo=None, num_q_samples=5000):
    ### STUDENTS START
    raise NotImplementedError()  # REPLACE ME
    ### STUDENTS END


In [ ]:
kl(model, xx=xx)

In [ ]:
sum([kl(model, xx=xx) for xx, oo, yy in iterate_data('train', max_examples=100)]) / 100

### Using the sampler

In [ ]:
class SamplingBayesAgent(DecisionAgent):
    """
    A version of BayesAgent that evaluates each plan by sampling.
    """
    def __init__(self, *args, num_samples=100, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_samples = num_samples

    def decision(self, *, xx, oo=None):
        # Efficiency tip: Collect a single set of samples and reuse it to evaluate each action in turn.  
        # Efficiency tip: Use a Counter to consolidate duplicate samples.
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END


Using our trained model, let's see whether it makes equally good decisions on `hamming_task`!

In [ ]:
# Repeated from before 
hamming_agent = BayesAgent(hamming_task, model)
hamming_agent.test(iterate_data('dev'))

In [ ]:
hamming_agent = SamplingBayesAgent(hamming_task, model, num_samples=20)
hamming_agent.test(iterate_data('dev'))

Was this faster?  Were the decisions as good?  What if you change the number of samples per decision?

**Discussion:** <span style='color:red'>FILL IN</span>

# The final test!

Throughout this homework we have implemented a number of different setups, and we have been tuning, debugging and developing these models using our *development* set.  Now that we have reached the end, choose a task setting that you like.  Choose **one** agent for that setting (including **one** model and **one** decision rule), and choose **one** training procedure that you use to train it on the training data.

In [ ]:
### STUDENTS START
### my_chosen_task = ????
### my_chosen_model = ????
### my_chosen_agent = ????
raise NotImplementedError()  # REPLACE ME
### STUDENTS END

my_chosen_trainer.train(my_chosen_agent, iterate_data('train'))

Now run your trained agent on the test data.  Only execute this cell once!  Don't go back and change your choices after you see the result!

In [ ]:
my_chosen_agent.test(iterate_data('test'))